In [166]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary
import matplotlib.pyplot as plt
from PIL import Image
import time

In [167]:
data_paths = {
    'train': './data/cassavaleafdata/train',
    'val': './data/cassavaleafdata/validation',
    'test': './data/cassavaleafdata/test'
}

def loader(path):
    return Image.open(path)

In [168]:
img_size = 150
BATCH_SIZE = 256 

train_transforms = transforms.Compose([
    transforms.Resize((150,  150)),
    transforms.ToTensor(),
])

print(train_transforms)

train_data = datasets.ImageFolder(
    root=data_paths['train'],
    loader=loader,
    transform=train_transforms 
)

valid_data = datasets.ImageFolder(
    root=data_paths['val'],
    transform = train_transforms
)

test_data = datasets.ImageFolder(
    root=data_paths['test'],
    transform = train_transforms
)

train_dataloader = data.DataLoader(
    train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_dataloader = data.DataLoader(
    valid_data,
    batch_size=BATCH_SIZE
)

test_dataloader = data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE
)

Compose(
    Resize(size=(150, 150), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)


In [169]:
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, padding='same'),
            nn.AvgPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=6, out_channels=16,kernel_size=5),
            nn.AvgPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(16*35*35, 120),
            nn.Linear(120, 84),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        return self.cnn(x)

In [170]:
summary(lenet_model, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 150, 150]             456
         AvgPool2d-2            [-1, 6, 75, 75]               0
              ReLU-3            [-1, 6, 75, 75]               0
            Conv2d-4           [-1, 16, 71, 71]           2,416
         AvgPool2d-5           [-1, 16, 35, 35]               0
              ReLU-6           [-1, 16, 35, 35]               0
           Flatten-7                [-1, 19600]               0
            Linear-8                  [-1, 120]       2,352,120
            Linear-9                   [-1, 84]          10,164
           Linear-10                    [-1, 5]             425
Total params: 2,365,581
Trainable params: 2,365,581
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 2.61
Params size (MB): 9.02
Estimat

In [171]:
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        predictions = model(inputs)

        loss = criterion(predictions, labels)
        losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print (
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [172]:
def evaluate(model, criterion, valid_dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):

            inputs, labels = inputs.to(device), labels.to(device)

            predictions = model(inputs)

            loss = criterion(predictions, labels)
            losses.append(loss.item())

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss
        

In [173]:
num_classes = len(train_data.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lenet_model = LeNetClassifier(num_classes)
lenet_model.to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 2e-4
optimizer = optim.Adam(lenet_model.parameters(), learning_rate)

num_epochs = 10
save_model = './model'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100

for epoch in range(1, num_epochs+1):
    epoch_start_time = time.time()

    train_acc, train_loss = train(lenet_model, optimizer, criterion, train_dataloader, device, epoch, log_interval=10)
    train_accs.append(train_acc)
    train_losses.append(train_loss)

    eval_acc, eval_loss = evaluate(lenet_model, criterion, valid_dataloader)
    eval_accs.append(eval_acc)
    eval_losses.append(eval_loss)

    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model_leaf.pt')

    print("-" * 59)
    print(
        "| End of epoch {:3d} | Time : {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} "
        "| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss
        )
    )
    print("-" * 59)

    lenet_model.load_state_dict(torch.load(save_model + '/lenet_model_leaf.pt', weight_only=True))
    lenet_model.eval()

| epoch   1 |    10/   23 batches | accuracy    0.430
| epoch   1 |    20/   23 batches | accuracy    0.468
-----------------------------------------------------------
| End of epoch   1 | Time : 67.15s | Train Accuracy    0.511 | Train Loss    1.393 | Valid Accuracy    0.470 | Valid Loss    1.498 
-----------------------------------------------------------
| epoch   2 |    10/   23 batches | accuracy    0.468
| epoch   2 |    20/   23 batches | accuracy    0.474
-----------------------------------------------------------
| End of epoch   2 | Time : 62.93s | Train Accuracy    0.471 | Train Loss    1.340 | Valid Accuracy    0.469 | Valid Loss    1.493 
-----------------------------------------------------------
| epoch   3 |    10/   23 batches | accuracy    0.480
| epoch   3 |    20/   23 batches | accuracy    0.479
-----------------------------------------------------------
| End of epoch   3 | Time : 58.74s | Train Accuracy    0.539 | Train Loss    1.293 | Valid Accuracy    0.493 | V

In [174]:
test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader)
test_acc, test_loss


(0.576657824933687, 1.2385087683796883)